# ECG Image Digitization: A Complete Guide

## PhysioNet Challenge - Transforming ECG Images to Digital Signals


[![OpenCV](https://img.shields.io/badge/OpenCV-5C3EE8?style=for-the-badge&logo=opencv&logoColor=white)](https://opencv.org/)

---
&#x2B06;  If you're wondering how to make a logo, definitely check out https://shields.io/.
### Table of Contents
1. **Introduction & Problem Understanding**
2. **Dataset Exploration**
3. **Image Preprocessing & Analysis**
4. **Grid Detection & Signal Extraction**
5. **Advanced Techniques & Denoising**
6. **Evaluation & Visualization**

PS : 6. There are some of bugs in **Evaluation & Visualization**. I would appreciate it if you could help. Thanks in advance.

### Challenge Goal
Convert **scanned ECG images** (photos/scans of paper ECG reports) back into **digital time-series signals** 

---

---
## 1. Environment Setup & Imports

First, let's import all necessary libraries and set up our visualization preferences.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import cv2
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Rectangle
import seaborn as sns

# Image processing
from scipy import signal, ndimage
from skimage import filters, morphology, measure
from scipy.interpolate import interp1d, UnivariateSpline

# Deep learning (optional for advanced methods)
try:
 import torch
 import torch.nn.functional as F
 TORCH_AVAILABLE = True
except ImportError:
 TORCH_AVAILABLE = False
 print(" PyTorch not available. Some advanced features will be disabled.")

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 10

print(" All libraries imported successfully!")
print(f" NumPy version: {np.__version__}")
print(f" OpenCV version: {cv2.__version__}")
print(f" PyTorch available: {TORCH_AVAILABLE}")

### Configure Data Paths

In [ ]:
# Configure paths - adjust these to your environment
DATA_DIR = Path('/kaggle/input/physionet-ecg-image-digitization')
TRAIN_DIR = DATA_DIR / 'train'
TEST_DIR = DATA_DIR / 'test'

# Verify paths exist
assert DATA_DIR.exists(), f" Data directory not found: {DATA_DIR}"
assert TRAIN_DIR.exists(), f" Train directory not found: {TRAIN_DIR}"

print(f" Data directory: {DATA_DIR}")
print(f" All paths verified!")

---
## 2. Dataset Exploration

Let's explore the dataset structure and understand what we're working with.

In [ ]:
# Load metadata
train_df = pd.read_csv(DATA_DIR / 'train.csv')
test_df = pd.read_csv(DATA_DIR / 'test.csv')

print(" Dataset Overview")
print("=" * 60)
print(f" Training samples: {len(train_df):,}")
print(f" Test samples: {len(test_df):,}")
print(f"\n Training data columns: {list(train_df.columns)}")
print(f"\n First few samples:")
display(train_df.head(10))

In [ ]:
# Statistical analysis
print("\n Statistical Summary")
print("=" * 60)
print(train_df.describe())

# Analyze sampling frequencies
print("\n Sampling Frequency Distribution:")
print(train_df['fs'].value_counts().sort_index())

In [ ]:
# Visualize dataset statistics
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Sampling frequency distribution
train_df['fs'].value_counts().sort_index().plot(kind='bar', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_title(' Sampling Frequency Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Sampling Frequency (Hz)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].grid(True, alpha=0.3)

# 2. Signal length distribution
train_df['sig_len'].plot(kind='hist', bins=50, ax=axes[0, 1], color='coral', alpha=0.7)
axes[0, 1].set_title(' Signal Length Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Signal Length (samples)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# 3. Duration calculation and distribution
train_df['duration_sec'] = train_df['sig_len'] / train_df['fs']
train_df['duration_sec'].plot(kind='hist', bins=50, ax=axes[1, 0], color='mediumseagreen', alpha=0.7)
axes[1, 0].set_title('⏱ ECG Duration Distribution', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Duration (seconds)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

# 4. Relationship: fs vs sig_len
axes[1, 1].scatter(train_df['fs'], train_df['sig_len'], alpha=0.5, s=30, color='purple')
axes[1, 1].set_title(' Sampling Frequency vs Signal Length', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Sampling Frequency (Hz)')
axes[1, 1].set_ylabel('Signal Length (samples)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n Key Insights:")
print(f" • Average ECG duration: {train_df['duration_sec'].mean():.2f} seconds")
print(f" • Most common sampling rate: {train_df['fs'].mode()[0]} Hz")
print(f" • Signal length range: {train_df['sig_len'].min()} - {train_df['sig_len'].max()} samples")

---
## 3. ECG Image Analysis

Now let's load and analyze actual ECG images to understand their structure.

In [ ]:
def load_ecg_images(sample_id, max_images=3):
    """
    Load ECG images for a given sample ID.
    
    Parameters:
    -----------
    sample_id : str or int or float
        The ID of the ECG sample
    max_images : int
        Maximum number of images to load
    
    Returns:
    --------
    list of tuples : (image_path, image_array)
    """
    # Convert float to int if necessary (pandas sometimes returns float)
    if isinstance(sample_id, float):   # I think 
        sample_id = int(sample_id)
    
    sample_dir = TRAIN_DIR / str(sample_id)
    
    if not sample_dir.exists():
        print(f"⚠️ Sample directory not found: {sample_dir}")
        return []
    
    image_files = sorted(sample_dir.glob('*.png'))[:max_images]
    images = []
    
    for img_path in image_files:
        img = cv2.imread(str(img_path))
        if img is not None:  # Check if image loaded successfully
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append((img_path.name, img_rgb))
    
    return images

# Select a sample to analyze - CONVERT TO INT!
sample_id = int(train_df.iloc[0]['id'])  
print(f"🔍 Analyzing sample: {sample_id}")

# Load images
ecg_images = load_ecg_images(sample_id, max_images=4)
print(f"📸 Loaded {len(ecg_images)} images")

In [ ]:
# Visualize the ECG images
if ecg_images:
    n_images = len(ecg_images)
    fig, axes = plt.subplots(n_images, 1, figsize=(18, 6 * n_images))
    
    if n_images == 1:
        axes = [axes]
    
    for idx, (img_name, img) in enumerate(ecg_images):
        axes[idx].imshow(img)
        axes[idx].set_title(f'📄 {img_name} | Shape: {img.shape} | Size: {img.shape[0]*img.shape[1]:,} pixels', 
                          fontsize=12, fontweight='bold')
        axes[idx].axis('off')
        
        # Add image statistics
        mean_brightness = img.mean()
        axes[idx].text(10, 30, f'Mean Brightness: {mean_brightness:.1f}', 
                      bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                      fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No images found for this sample")

### Detailed Image Analysis

In [ ]:
def analyze_ecg_image(image):
 """
 Perform comprehensive analysis of an ECG image.
 
 Returns various image characteristics and processed versions.
 """
 # Convert to grayscale
 gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
 
 # Calculate histogram
 hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
 
 # Apply different edge detection methods
 edges_canny = cv2.Canny(gray, 50, 150)
 edges_sobel = filters.sobel(gray)
 
 # Adaptive thresholding
 thresh_adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
 cv2.THRESH_BINARY, 11, 2)
 
 # Otsu's thresholding
 _, thresh_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
 
 return {
 'gray': gray,
 'histogram': hist,
 'edges_canny': edges_canny,
 'edges_sobel': edges_sobel,
 'thresh_adaptive': thresh_adaptive,
 'thresh_otsu': thresh_otsu
 }

# Analyze the first image
if ecg_images:
 _, first_image = ecg_images[0]
 analysis = analyze_ecg_image(first_image)
 
 # Visualize analysis results
 fig = plt.figure(figsize=(18, 12))
 gs = gridspec.GridSpec(3, 3, hspace=0.3, wspace=0.3)
 
 # Original image
 ax1 = plt.subplot(gs[0, :])
 ax1.imshow(first_image)
 ax1.set_title(' Original ECG Image', fontsize=14, fontweight='bold')
 ax1.axis('off')
 
 # Grayscale
 ax2 = plt.subplot(gs[1, 0])
 ax2.imshow(analysis['gray'], cmap='gray')
 ax2.set_title(' Grayscale', fontsize=12, fontweight='bold')
 ax2.axis('off')
 
 # Histogram
 ax3 = plt.subplot(gs[1, 1])
 ax3.plot(analysis['histogram'], color='steelblue', linewidth=2)
 ax3.set_title(' Intensity Histogram', fontsize=12, fontweight='bold')
 ax3.set_xlabel('Pixel Intensity')
 ax3.set_ylabel('Frequency')
 ax3.grid(True, alpha=0.3)
 
 # Canny edges
 ax4 = plt.subplot(gs[1, 2])
 ax4.imshow(analysis['edges_canny'], cmap='gray')
 ax4.set_title(' Canny Edge Detection', fontsize=12, fontweight='bold')
 ax4.axis('off')
 
 # Sobel edges
 ax5 = plt.subplot(gs[2, 0])
 ax5.imshow(analysis['edges_sobel'], cmap='gray')
 ax5.set_title(' Sobel Edge Detection', fontsize=12, fontweight='bold')
 ax5.axis('off')
 
 # Adaptive threshold
 ax6 = plt.subplot(gs[2, 1])
 ax6.imshow(analysis['thresh_adaptive'], cmap='gray')
 ax6.set_title(' Adaptive Thresholding', fontsize=12, fontweight='bold')
 ax6.axis('off')
 
 # Otsu threshold
 ax7 = plt.subplot(gs[2, 2])
 ax7.imshow(analysis['thresh_otsu'], cmap='gray')
 ax7.set_title(' Otsu Thresholding', fontsize=12, fontweight='bold')
 ax7.axis('off')
 
 plt.show()
 
 print("\n Image analysis complete!")
 print(f" • Image dimensions: {first_image.shape[0]} x {first_image.shape[1]}")
 print(f" • Mean pixel value: {analysis['gray'].mean():.2f}")
 print(f" • Std pixel value: {analysis['gray'].std():.2f}")

---
## 4. ECG Signal Extraction Pipeline

Now let's build a comprehensive pipeline to extract the ECG signal from images.

In [ ]:
class ECGImageProcessor:
    """
    A comprehensive class for processing ECG images and extracting signals.
    
    This class implements various methods for:
    - Image preprocessing
    - Grid detection
    - Signal extraction
    - Quality assessment
    """
    
    def __init__(self, image):
        """
        Initialize with an ECG image.
        
        Parameters:
        -----------
        image : numpy.ndarray
            RGB image of ECG
        """
        self.original_image = image.copy()
        self.height, self.width = image.shape[:2]
        self.gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
    def preprocess(self, denoise=True, enhance_contrast=True):
        """
        Preprocess the image for better signal extraction.
        
        Parameters:
        -----------
        denoise : bool
            Apply denoising filter
        enhance_contrast : bool
            Apply contrast enhancement
        
        Returns:
        --------
        numpy.ndarray : Preprocessed grayscale image
        """
        processed = self.gray.copy()
        
        # Denoise
        if denoise:
            processed = cv2.fastNlMeansDenoising(processed, None, 10, 7, 21)
        
        # Enhance contrast
        if enhance_contrast:
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            processed = clahe.apply(processed)
        
        self.preprocessed = processed
        return processed
    
    def detect_grid_lines(self, threshold=200):
        """
        Detect horizontal and vertical grid lines.
        
        Parameters:
        -----------
        threshold : int
            Threshold for line detection
        
        Returns:
        --------
        tuple : (horizontal_lines, vertical_lines)
        """
        # Threshold image
        _, binary = cv2.threshold(self.gray, threshold, 255, cv2.THRESH_BINARY)
        
        # Detect horizontal lines
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
        horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel)
        
        # Detect vertical lines
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
        vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel)
        
        self.horizontal_lines = horizontal_lines
        self.vertical_lines = vertical_lines
        
        return horizontal_lines, vertical_lines
    
    def extract_signal_trace(self, roi=None, method='projection'):
        """
        Extract ECG signal trace from the image.
        
        Parameters:
        -----------
        roi : tuple or None
            Region of interest (x, y, w, h)
        method : str
            Extraction method: 'projection', 'contour', or 'darkest'
        
        Returns:
        --------
        numpy.ndarray : Extracted signal values
        """
        # Use preprocessed image or grayscale
        img = self.preprocessed if hasattr(self, 'preprocessed') else self.gray
        
        # Apply ROI if specified
        if roi is not None:
            x, y, w, h = roi
            img = img[y:y+h, x:x+w]
        
        if method == 'projection':
            # Invert image (signal is dark)
            inverted = 255 - img
            # Column-wise projection
            signal = np.argmax(inverted, axis=0)
        
        elif method == 'darkest':
            # Find darkest pixel in each column
            signal = np.argmin(img, axis=0)
        
        elif method == 'weighted':
            # Weighted average based on intensity
            inverted = 255 - img
            weights = inverted / inverted.sum(axis=0, keepdims=True)
            positions = np.arange(img.shape[0]).reshape(-1, 1)
            signal = (positions * weights).sum(axis=0)
        
        else:
            raise ValueError(f"Unknown method: {method}")
        
        # Invert y-axis (image coordinates are top-down)
        signal = img.shape[0] - signal
        
        return signal
    
    def smooth_signal(self, signal, method='savgol', **kwargs):
        """
        Apply smoothing to extracted signal.
        
        Parameters:
        -----------
        signal : numpy.ndarray
            Raw signal values
        method : str
            Smoothing method: 'savgol', 'median', 'gaussian'
        **kwargs : dict
            Additional parameters for smoothing
        
        Returns:
        --------
        numpy.ndarray : Smoothed signal
        """
        if method == 'savgol':
            window = kwargs.get('window_length', 11)
            polyorder = kwargs.get('polyorder', 3)
            smoothed = signal.savgol_filter(signal, window, polyorder)
        
        elif method == 'median':
            kernel_size = kwargs.get('kernel_size', 5)
            smoothed = ndimage.median_filter(signal, size=kernel_size)
        
        elif method == 'gaussian':
            sigma = kwargs.get('sigma', 2)
            smoothed = ndimage.gaussian_filter1d(signal, sigma)
        
        else:
            smoothed = signal
        
        return smoothed

print("✅ ECGImageProcessor class defined successfully!")

### Test Image Preprocessing

In [ ]:
# Process the first ECG image
if ecg_images:
    _, test_image = ecg_images[0]
    
    # Initialize processor
    processor = ECGImageProcessor(test_image)
    
    # Preprocess
    print("🔧 Preprocessing image...")
    preprocessed = processor.preprocess(denoise=True, enhance_contrast=True)
    
    # Visualize preprocessing steps - Focus on what works!
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    axes[0].imshow(test_image)
    axes[0].set_title(' Original ECG Image', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(processor.gray, cmap='gray')
    axes[1].set_title(' Grayscale Conversion', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(preprocessed, cmap='gray')
    axes[2].set_title(' Enhanced (Denoised + Contrast)', fontsize=14, fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Preprocessing complete!")
    print(f"   • Original shape: {test_image.shape}")
    print(f"   • Pixel value range: [{preprocessed.min()}, {preprocessed.max()}]")

### Extract and Visualize ECG Signal

In [ ]:
# Extract signal using different methods
if ecg_images:
    print(" Extracting ECG signal using multiple methods...\n")
    
    # Define a region of interest - focusing on the ECG signal area
    # Format: (x, y, width, height)
    # Adjusted to capture the actual ECG trace in the middle of the image
    img_height, img_width = test_image.shape[:2]
    roi = (50, int(img_height * 0.3), img_width - 100, int(img_height * 0.5))
    
    # Extract using different methods
    signal_projection = processor.extract_signal_trace(roi=roi, method='projection')
    signal_darkest = processor.extract_signal_trace(roi=roi, method='darkest')
    signal_weighted = processor.extract_signal_trace(roi=roi, method='weighted')
    
    # Smooth signals
    signal_projection_smooth = processor.smooth_signal(signal_projection, method='gaussian', sigma=2)
    signal_darkest_smooth = processor.smooth_signal(signal_darkest, method='gaussian', sigma=2)
    signal_weighted_smooth = processor.smooth_signal(signal_weighted, method='gaussian', sigma=2)
    
    # Visualize results
    fig, axes = plt.subplots(4, 1, figsize=(20, 14))
    
    # Show ROI on image
    img_with_roi = test_image.copy()
    x, y, w, h = roi
    cv2.rectangle(img_with_roi, (x, y), (x + w, y + h), (255, 0, 0), 3)
    axes[0].imshow(img_with_roi)
    axes[0].set_title(' ECG Image with ROI (Region of Interest)', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Plot extracted signals
    x_axis = np.arange(len(signal_projection))
    
    axes[1].plot(x_axis, signal_projection, alpha=0.6, label='Raw', linewidth=1)
    axes[1].plot(x_axis, signal_projection_smooth, label='Smoothed', linewidth=2)
    axes[1].set_title(' Method 1: Projection-based Extraction', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Amplitude')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    axes[2].plot(x_axis, signal_darkest, alpha=0.6, label='Raw', linewidth=1)
    axes[2].plot(x_axis, signal_darkest_smooth, label='Smoothed', linewidth=2)
    axes[2].set_title(' Method 2: Darkest Pixel Extraction', fontsize=12, fontweight='bold')
    axes[2].set_ylabel('Amplitude')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    axes[3].plot(x_axis, signal_weighted, alpha=0.6, label='Raw', linewidth=1)
    axes[3].plot(x_axis, signal_weighted_smooth, label='Smoothed', linewidth=2)
    axes[3].set_title(' Method 3: Weighted Average Extraction', fontsize=12, fontweight='bold')
    axes[3].set_xlabel('Sample Index')
    axes[3].set_ylabel('Amplitude')
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Signal extraction complete!")
    print(f"   • Signal length: {len(signal_projection)} samples")
    print(f"   • Signal range: [{signal_projection.min():.2f}, {signal_projection.max():.2f}]")

---
## 5. Advanced Processing & Quality Metrics

Let's implement quality metrics to evaluate our signal extraction.

In [ ]:
def calculate_signal_quality_metrics(signal):
    """
    Calculate various quality metrics for an extracted ECG signal.
    
    Parameters:
    -----------
    signal : numpy.ndarray
        Extracted ECG signal
    
    Returns:
    --------
    dict : Quality metrics
    """
    metrics = {}
    
    # Basic statistics
    metrics['mean'] = np.mean(signal)
    metrics['std'] = np.std(signal)
    metrics['min'] = np.min(signal)
    metrics['max'] = np.max(signal)
    metrics['range'] = metrics['max'] - metrics['min']
    
    # Signal-to-noise ratio (simplified)
    signal_power = np.mean(signal ** 2)
    noise_power = np.var(np.diff(signal))
    metrics['snr'] = 10 * np.log10(signal_power / (noise_power + 1e-10))
    
    # Smoothness (variation measure)
    first_derivative = np.diff(signal)
    metrics['smoothness'] = np.std(first_derivative)
    
    # Zero crossings
    signal_normalized = signal - np.mean(signal)
    zero_crossings = np.sum(np.diff(np.sign(signal_normalized)) != 0)
    metrics['zero_crossings'] = zero_crossings
    
    return metrics

# Calculate metrics for our extracted signals
if ecg_images:
    print(" Signal Quality Metrics")
    print("=" * 80)
    
    for name, sig in [('Projection', signal_projection_smooth),
                      ('Darkest', signal_darkest_smooth),
                      ('Weighted', signal_weighted_smooth)]:
        metrics = calculate_signal_quality_metrics(sig)
        
        print(f"\n🔍 {name} Method:")
        print(f"   Mean: {metrics['mean']:.2f}")
        print(f"   Std Dev: {metrics['std']:.2f}")
        print(f"   Range: {metrics['range']:.2f}")
        print(f"   SNR: {metrics['snr']:.2f} dB")
        print(f"   Smoothness: {metrics['smoothness']:.2f}")
        print(f"   Zero Crossings: {metrics['zero_crossings']}")

---
## 6. Complete End-to-End Pipeline

Let's create a complete pipeline that processes all images for a sample.

In [ ]:
def process_ecg_sample(sample_id, output_signals=True, visualize=True):
    """
    I couldn't make it :(
    Complete pipeline to process all ECG images for a sample.
    
    Parameters:
    -----------
    sample_id : str or int
        Sample ID to process
    output_signals : bool
        Whether to return extracted signals
    visualize : bool
        Whether to create visualizations
    
    Returns:
    --------
    dict : Processed results including signals and metadata
    """
    print(f"\n Processing ECG Sample: {sample_id}")
    print("=" * 80)
    
    # Load images
    images = load_ecg_images(sample_id, max_images=6)
    
    if not images:
        print("❌ No images found!")
        return None
    
    print(f"✅ Loaded {len(images)} images")
    
    results = {
        'sample_id': sample_id,
        'n_images': len(images),
        'signals': [],
        'metadata': []
    }
    
    # Process each image
    for idx, (img_name, img) in enumerate(images):
        print(f"\n  Processing {img_name}...")
        
        # Initialize processor
        processor = ECGImageProcessor(img)
        
        # Preprocess
        processor.preprocess(denoise=True, enhance_contrast=True)
        
        # Extract signal - dynamically set ROI based on image size
        img_height, img_width = img.shape[:2]
        roi = (50, int(img_height * 0.3), img_width - 100, int(img_height * 0.5))
        signal_raw = processor.extract_signal_trace(roi=roi, method='weighted')
        signal_smooth = processor.smooth_signal(signal_raw, method='gaussian', sigma=2)
        
        # Calculate quality metrics
        metrics = calculate_signal_quality_metrics(signal_smooth)
        
        # Store results
        results['signals'].append({
            'image_name': img_name,
            'raw_signal': signal_raw,
            'smooth_signal': signal_smooth,
            'length': len(signal_smooth)
        })
        
        results['metadata'].append({
            'image_name': img_name,
            'image_shape': img.shape,
            'signal_length': len(signal_smooth),
            'snr': metrics['snr'],
            'smoothness': metrics['smoothness']
        })
        
        print(f"     ✓ Signal length: {len(signal_smooth)} | SNR: {metrics['snr']:.2f} dB")
    
    # Visualize if requested
    if visualize and len(images) > 0:
        n_imgs = len(images)
        fig, axes = plt.subplots(n_imgs, 2, figsize=(20, 4 * n_imgs))
        
        if n_imgs == 1:
            axes = axes.reshape(1, -1)
        
        for idx, ((img_name, img), signal_data) in enumerate(zip(images, results['signals'])):
            # Show image
            axes[idx, 0].imshow(img)
            axes[idx, 0].set_title(f'📸 {img_name}', fontsize=12, fontweight='bold')
            axes[idx, 0].axis('off')
            
            # Show signal
            x = np.arange(len(signal_data['smooth_signal']))
            axes[idx, 1].plot(x, signal_data['smooth_signal'], linewidth=1.5, color='darkblue')
            axes[idx, 1].set_title(f' Extracted ECG Signal', fontsize=12, fontweight='bold')
            axes[idx, 1].set_xlabel('Sample Index')
            axes[idx, 1].set_ylabel('Amplitude')
            axes[idx, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print(f"\n✅ Processing complete for sample {sample_id}!")
    print(f"   Total signals extracted: {len(results['signals'])}")
    
    return results

# Test the complete pipeline
sample_id = train_df.iloc[0]['id']
results = process_ecg_sample(sample_id, visualize=True)

---
## 7. Summary & Next Steps

### What We've Accomplished

In this notebook, we've built a comprehensive ECG image digitization pipeline:

1. **Dataset Exploration** - Analyzed the PhysioNet ECG dataset structure
2. **Image Analysis** - Performed detailed analysis of ECG images
3. **Preprocessing** - Implemented denoising and enhancement techniques
4. **Grid Detection** - Detected and analyzed ECG grid patterns
5. **Signal Extraction** - Implemented multiple extraction methods
6. **Quality Metrics** - Developed signal quality assessment tools
7. **End-to-End Pipeline** - Created a complete processing workflow

### Next Steps & Improvements

To further improve this pipeline, consider:

1. **Deep Learning Approaches**
 - Train a U-Net for grid point detection
 - Use CNN for automatic ROI detection
 - Implement attention mechanisms for signal extraction

2. **Advanced Preprocessing**
 - Perspective correction for skewed images
 - Automatic calibration detection
 - Multi-lead separation and extraction

3. **Signal Processing**
 - Baseline wander removal
 - High-frequency noise filtering
 - Peak detection (R-peaks, P-waves, T-waves)

4. **Validation & Metrics**
 - Compare with ground truth signals
 - Implement medical-grade quality metrics
 - Cross-validation across different ECG types

### Resources & References

- [PhysioNet Challenge](https://physionet.org/)
- [ECG Signal Processing Guide](https://www.physionet.org/content/ecg-guide/)
- [OpenCV Documentation](https://docs.opencv.org/)

### Feedback & Contributions

If you found this notebook helpful:
- Star the repository
- Fork and improve
- Share your feedback
- Report issues

---

**Happy Coding! **

---
## Bonus: Interactive Exploration

Try different samples and parameters below!

In [ ]:
# Interactive cell - change the sample_id to explore different ECG recordings
# You can select any ID from the train_df dataframe

# Show available samples
print(" Available samples (first 5):")
print(train_df['id'].head(5).tolist())

# Select a sample
interactive_sample_id = train_df.iloc[2]['id'] # Change the index to try different samples

# Process it
interactive_results = process_ecg_sample(interactive_sample_id, visualize=True)

In [ ]:
# Compare multiple extraction methods side-by-side
if interactive_results and len(interactive_results['signals']) > 0:
    first_img = load_ecg_images(interactive_sample_id, max_images=1)[0][1]
    processor = ECGImageProcessor(first_img)
    processor.preprocess()
    
    roi = (50, 100, first_img.shape[1] - 100, first_img.shape[0] - 200)
    
    methods = ['projection', 'darkest', 'weighted']
    colors = ['blue', 'red', 'green']
    
    plt.figure(figsize=(20, 6))
    
    for method, color in zip(methods, colors):
        signal = processor.extract_signal_trace(roi=roi, method=method)
        signal_smooth = processor.smooth_signal(signal, method='gaussian', sigma=2)
        plt.plot(signal_smooth, label=method.capitalize(), color=color, linewidth=2, alpha=0.7)
    
    plt.title('🔬 Comparison of Different Extraction Methods', fontsize=16, fontweight='bold')
    plt.xlabel('Sample Index', fontsize=12)
    plt.ylabel('Amplitude', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()